# Tutorial 2.2: Inputs, Files, and Scans

## 2.2.0: Python Imports

In [2]:
%load_ext autoreload
%autoreload 2

Please import:

In [47]:
from mbfit.utils.filesystem import Directory

from mbfit.system.system_like import BondType
from mbfit.system.definition import BasicFragmentDefinition, BasicCompoundDefinition, AtomDefinition, BondDefinition
from mbfit.system.initializer import VSEPRInitializer
from mbfit.system.descriptor import fragment_definition_to_xyz, compound_definition_to_xyz, compound_to_xyz
from mbfit.system.parser import xyz_to_compound_definition

## 2.2.1 Directory Structure & File Paths

In this section, we we will setup the directory structure and file paths we will use in this notebook. Throughout this notebook, the following files will be created:

In [7]:
tutorial_2_workdir = Directory("Tutorial_2_workdir")
tutorial_2_workdir.create()

In [10]:
definitions_dir = tutorial_2_workdir.sub_directory("definitions")
definitions_dir.create()

In [11]:
fragment_CH3NH2_definition_path = definitions_dir.file("fragment_CH3NH2.def")
fragment_H2O_definition_path = definitions_dir.file("fragment_H2O.def")

In [44]:
monomer_CH3NH2_definition_path = definitions_dir.file("monomer_CH3NH2.def")
monomer_H2O_definition_path = definitions_dir.file("monomer_H2O.def")
dimer_CH3NH2_H2O_definition_path = definitions_dir.file("dimer_CH3NH2_H2O.def")
trimer_CH3NH2_H2O_H2O_definition_path = definitions_dir.file("trimer_CH3NH2_H2O_H2O.def")

In [13]:
structures_dir = tutorial_2_workdir.sub_directory("structures")
structures_dir.create()

In [14]:
monomer_CH3NH2_initial_structure_path = structures_dir.file("monomer_CH3NH2_initial_structure.xyz")
monomer_H2O_initial_structure_path = structures_dir.file("monomer_H2O_initial_structure.xyz")

monomer_CH3NH2_minimum_structure_path = structures_dir.file("monomer_CH3NH2_minimum_structure.xyz")
monomer_H2O_minimum_structure_path = structures_dir.file("monomer_H2O_minimum_structure.xyz")

In [15]:
logs_dir = tutorial_2_workdir.sub_directory("logs")
logs_dir.create()

## 2.2.x Create `.def` definition files

First, we will create the two fragment definition files. First, the definition for the CH3NH2 fragment:

In [29]:
fragment_CH3NH2_definition = BasicFragmentDefinition()
CA  = fragment_CH3NH2_definition.add_atom(AtomDefinition("C", "A"))
HB1 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "B"))
HB2 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "B"))
HB3 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "B"))
NC  = fragment_CH3NH2_definition.add_atom(AtomDefinition("N", "C"))
HD1 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "D"))
HD2 = fragment_CH3NH2_definition.add_atom(AtomDefinition("H", "D"))
fragment_CH3NH2_definition.add_bond(CA, HB1, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(CA, HB2, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(CA, HB3, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(CA, NC, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(NC, HD1, BondDefinition(BondType.SINGLE))
fragment_CH3NH2_definition.add_bond(NC, HD2, BondDefinition(BondType.SINGLE))

fragment_definition_to_xyz.writer(fragment_CH3NH2_definition_path, fragment_CH3NH2_definition)

The definition for the H2O fragment:

In [33]:
fragment_H2O_definition = BasicFragmentDefinition()
OE  = fragment_H2O_definition.add_atom(AtomDefinition("O", "E"))
HF1 = fragment_H2O_definition.add_atom(AtomDefinition("H", "F"))
HF2 = fragment_H2O_definition.add_atom(AtomDefinition("H", "F"))
fragment_H2O_definition.add_bond(OE, HF1, BondDefinition(BondType.SINGLE))
fragment_H2O_definition.add_bond(OE, HF2, BondDefinition(BondType.SINGLE))

fragment_definition_to_xyz.writer(fragment_H2O_definition_path, fragment_H2O_definition)

Next, we will make three n-mer definitions.

First, the CH3NH2 monomer, which has a single CH3NH2 fragment.

In [36]:
monomer_CH3NH2_definition = BasicCompoundDefinition()
monomer_CH3NH2_definition.add_fragment(fragment_CH3NH2_definition)

compound_definition_to_xyz.writer(monomer_CH3NH2_definition_path, monomer_CH3NH2_definition)

First, the H2O monomer, which has a single CH3NH2 fragment.

In [45]:
monomer_H2O_definition = BasicCompoundDefinition()
monomer_H2O_definition.add_fragment(fragment_H2O_definition)

compound_definition_to_xyz.writer(monomer_H2O_definition_path, monomer_H2O_definition)

Next, the CH3NH2 -- H2O dimer, with two fragments:

In [37]:
dimer_CH3NH2_H2O_definition = BasicCompoundDefinition()
dimer_CH3NH2_H2O_definition.add_fragment(fragment_CH3NH2_definition)
dimer_CH3NH2_H2O_definition.add_fragment(fragment_H2O_definition)

compound_definition_to_xyz.writer(dimer_CH3NH2_H2O_definition_path, dimer_CH3NH2_H2O_definition)

Finally, the CH3NH2 -- H2O -- H2O trimer, with three fragments:

In [38]:
trimer_CH3NH2_H2O_H2O_definition = BasicCompoundDefinition()
trimer_CH3NH2_H2O_H2O_definition.add_fragment(fragment_CH3NH2_definition)
trimer_CH3NH2_H2O_H2O_definition.add_fragment(fragment_H2O_definition)
trimer_CH3NH2_H2O_H2O_definition.add_fragment(fragment_H2O_definition)

compound_definition_to_xyz.writer(trimer_CH3NH2_H2O_H2O_definition_path, trimer_CH3NH2_H2O_H2O_definition)

We will use these definition files throughout Tutorial 2.

## 2.2.x Geometry Initialization

Next, we need a reasonable initial guess for the structure of our monomer. The guess need only be good enough that it will covnerge to the actual minimum structure when we perform a minimization with DFT. MB-Fit contains the class `VESPRInitializer`, which will make a reasonable guess about the structure of a molecule based on its bonding topology. Let's use `VESPRInitializer` to create an initial structure file:

In [48]:
definition = xyz_to_compound_definition.reader(monomer_CH3NH2_definition_path)
system_initializer = VSEPRInitializer()
initial_structure = system_initializer(definition)
compound_to_xyz.writer(monomer_CH3NH2_initial_structure_path, initial_structure)

Initializing compound C1H3N1H2
System initialization complete!


In [49]:
definition = xyz_to_compound_definition.reader(monomer_H2O_definition_path)
system_initializer = VSEPRInitializer()
initial_structure = system_initializer(definition)
compound_to_xyz.writer(monomer_H2O_initial_structure_path, initial_structure)

Initializing compound O1H2
System initialization complete!


## 2.2.x Geometry Minimization

That guess is looking a little rough, lets optimize it using electronic structure theory. While the PEF will be developed at the CCSD(T) level of theory, its not important that the geometry optimization be at the same level of theory as the reference energies will be calculated at, so lets use something cheaper like a high-quality DFT functional.

In [ ]:
job = job_runner.queue_job(
f"""
calculator = QChemCalculator("wb97m-v", "aug-cc-pvTZ", logs_directory="{logs_dir}")
initial_structure = xyz_reader("{monomer_CH3NH2_initial_structure_path}")
optimized_structure, optimized_energy, optimization_log_path = calculator.energy(initial_structure)
xyz_writer("{monomer_CH3NH2_minimum_structure_path}", optimized_structure)
"""
)

job.wait_until_running()
job.wait_until_done()

In [ ]:
job = job_runner.queue_job(
f"""
calculator = QChemCalculator("wb97m-v", "aug-cc-pvTZ", logs_directory="{logs_dir}")
initial_structure = xyz_reader("{monomer_H2O_initial_structure_path}")
optimized_structure, optimized_energy, optimization_log_path = calculator.energy(initial_structure)
xyz_writer("{monomer_CH3NH2_minimum_structure_path}", optimized_structure)
"""
)

job.wait_until_running()
job.wait_until_done()

## 2.2.2 Two-Body Scans

Now we make a few two-body scans. We will use these during future steps to validate the quality of the 2-body parts of the PEF.

Lets generate a few 2-body CH$_3$-NH$_2$ -- H$_2$O scans. We will make three scans:
* H-bond between H on -NH$_2$ and O in H$_2$O
* H-bond between H on H$_2$O and N in -NH$_2$
* H from -CH$_3$ approaching O in H$_2$O

In [ ]:
dimer_CH3NH2_H2O_definition = ...

transformer1 = SystemTransformer(monomer1)
transformer2 = SystemTransformer(monomer2)
transformer1.align(vectors_to_align=(...))
transformer2.align(vectors_to_align=(...))

scan = [...]

In [ ]:
dimer_CH3NH2_H2O_definition = ...

transformer1 = SystemTransformer(monomer1)
transformer2 = SystemTransformer(monomer2)
transformer1.align(vectors_to_align=(...))
transformer2.align(vectors_to_align=(...))

scan = [...]

In [ ]:
dimer_CH3NH2_H2O_definition = ...

transformer1 = SystemTransformer(monomer1)
transformer2 = SystemTransformer(monomer2)
transformer1.align(vectors_to_align=(...))
transformer2.align(vectors_to_align=(...))

scan = [...]

Now, lets perform the reference calculations for the scans.

## 2.2.3 Three-Body Scans

Now we make a few three-body scans. We will use these during future steps to validate the quality of the 2-body parts of the PEF.

Lets also consider a few 3-body structures. We will make two scans:
* -NH$_2$ donating 2 H-bonds to 2 H$_2$Os, where one H$_2$O moves away.
* -NH$_2$ donating 1 H-bond and accepting 1 H-bond, where the H$_2$O that is donating an H-bond moves away.
* -NH$_2$ donating 1 H-bond and accepting 1 H-bond, where the H$_2$O that is accepting an H-bond moves away.

In [ ]:
dimer_CH3NH2_H2O_definition = ...

transformer1 = SystemTransformer(monomer1)
transformer2 = SystemTransformer(monomer2)
transformer1.align(vectors_to_align=(...))
transformer2.align(vectors_to_align=(...))

scan = [...]

In [ ]:
dimer_CH3NH2_H2O_definition = ...

transformer1 = SystemTransformer(monomer1)
transformer2 = SystemTransformer(monomer2)
transformer1.align(vectors_to_align=(...))
transformer2.align(vectors_to_align=(...))

scan = [...]

In [ ]:
dimer_CH3NH2_H2O_definition = ...

transformer1 = SystemTransformer(monomer1)
transformer2 = SystemTransformer(monomer2)
transformer1.align(vectors_to_align=(...))
transformer2.align(vectors_to_align=(...))

scan = [...]